In [ ]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense, SpatialDropout1D
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn import preprocessing

import pandas as pd
import numpy as np
import re

from numpy import array
from numpy import asarray
from numpy import zeros

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords

from keras.utils.np_utils import to_categorical

from keras.utils.vis_utils import plot_model

import matplotlib.pyplot as plt

In [ ]:
STOPWORDS = set(stopwords.words('english'))
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')

In [ ]:
def preprocess_text(sen):

    stemmer = WordNetLemmatizer()

   # Remove all the special characters
    document = re.sub(r'\W', ' ', str(sen))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 

    document = REPLACE_BY_SPACE_RE.sub(' ', document)
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Converting to Lowercase
    document = document.lower()
  
    document = document.split()

    document = ' '.join(word for word in document if word not in STOPWORDS) # remove stopwors from text

    # Lemmatization
  
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document) 
    
    return document

In [ ]:
def getModel(maxlen, embedding_matrix):
  input_1 = Input(shape=(maxlen,))
  input_2 = Input(shape=(4,)) 

  embedding_layer = Embedding(vocab_size, 50, weights=[embedding_matrix], trainable=False)(input_1)
  LSTM_Layer_1 = LSTM(70, activation='tanh')(embedding_layer)

  dense_layer_1 = Dense(50, activation='tanh')(input_2)
  dense_layer_2 = Dense(20, activation='tanh')(dense_layer_1)

  concat_layer = Concatenate()([LSTM_Layer_1, dense_layer_2])
  dense_layer_3 = Dense(10, activation='tanh')(concat_layer)
  output = Dense(4, activation='softmax')(dense_layer_3)
  
  return Model(inputs=[input_1, input_2], outputs=output)

In [ ]:
def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy

    Variables:
        weights: numpy array of shape (C,) where C is the number of classes

    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """

    weights = K.variable(weights)

    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss

    return loss

In [ ]:
train_path = 'Training_set.csv'

col_names = ['triggerTitle','triggerChannelTitle','actionChannelTitle','actionTitle','title', 'desc', 'target']
train_final = pd.read_csv(train_path,skiprows=1,sep=',',names=col_names,encoding = "ISO-8859-1")

In [ ]:
test_path = 'Test_set.csv'

col_names = ['triggerTitle','triggerChannelTitle','actionChannelTitle','actionTitle','title', 'desc', 'target']
test_final = pd.read_csv(test_path,skiprows=1,sep=';',names=col_names,encoding = "ISO-8859-1")

In [ ]:
train_df_extract = train_final[["title", "desc"]]

title_desc_concat = []

for i in range(0,len(train_df_extract)):
  sentence = str(train_df_extract.iloc[i][0]) + ". " + str(train_df_extract.iloc[i][1] + ".")
  title_desc_concat.append(sentence)

In [ ]:
del train_final['title']
del train_final['desc']

target_column = train_final[["target"]]
del train_final['target']

train_final['sentence'] = title_desc_concat
train_final['target'] = target_column

# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

# Encode labels in column 'species'.
train_final['triggerTitle'] = label_encoder.fit_transform(train_final['triggerTitle'])
train_final['triggerChannelTitle'] = label_encoder.fit_transform(train_final['triggerChannelTitle'])
train_final['actionChannelTitle'] = label_encoder.fit_transform(train_final['actionChannelTitle'])
train_final['actionTitle'] = label_encoder.fit_transform(train_final['actionTitle'])

In [ ]:
X_train = train_final.drop('target', axis=1)

y_train = train_final['target']

In [ ]:
test_df_extract = test_final[["title", "desc"]]

title_desc_concat = []

for i in range(0,len(test_df_extract)):
  sentence = str(test_df_extract.iloc[i][0]) + ". " + str(test_df_extract.iloc[i][1])
  title_desc_concat.append(sentence)

In [ ]:
from sklearn import preprocessing

del test_final['title']
del test_final['desc']

target_column = test_final[["target"]]
del test_final['target']

test_final['sentence'] = title_desc_concat
test_final['target'] = target_column

# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

# Encode labels in column 'species'.
test_final['triggerTitle'] = label_encoder.fit_transform(test_final['triggerTitle'])
test_final['triggerChannelTitle'] = label_encoder.fit_transform(test_final['triggerChannelTitle'])
test_final['actionChannelTitle'] = label_encoder.fit_transform(test_final['actionChannelTitle'])
test_final['actionTitle'] = label_encoder.fit_transform(test_final['actionTitle'])

In [ ]:
X_test = test_final.drop('target', axis=1)

y_test = test_final['target']

In [ ]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
X1_train = []
sentences = list(X_train["sentence"])
for sen in sentences:
    X1_train.append(preprocess_text(sen))

In [ ]:
X1_test = []
sentences = list(X_test["sentence"])
for sen in sentences:
    X1_test.append(preprocess_text(sen))

In [ ]:
tokenizer = Tokenizer(lower=True, num_words=None)
tokenizer.fit_on_texts(X1_train)

X1_train = tokenizer.texts_to_sequences(X1_train)
X1_test = tokenizer.texts_to_sequences(X1_test)

vocab_size = len(tokenizer.word_index) + 1

maxlen = 50

X1_train = pad_sequences(X1_train, padding='post', maxlen=maxlen)
X1_test = pad_sequences(X1_test, padding='post', maxlen=maxlen)

In [ ]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()

glove_file = open('Glove/glove.6B.50d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions

glove_file.close()

In [ ]:
embedding_matrix = zeros((vocab_size, 50))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [ ]:
X2_train = X_train[['triggerTitle', 'triggerChannelTitle', 'actionChannelTitle','actionTitle']].values
X2_test = X_test[['triggerTitle', 'triggerChannelTitle', 'actionChannelTitle','actionTitle']].values

In [ ]:
# STRATIFIES K-FOLD CROSS VALIDATION { 4-fold }

splits = 4

# Create StratifiedKFold object.
skf = StratifiedKFold(n_splits=splits, shuffle=True, random_state=1)

class_weights_list = []

for train_index, test_index in skf.split(X_train, y_train):
    count = count + 1
    x_train1_fold, x_test1_fold = X1_train.iloc[train_index], X1_train.iloc[test_index]
    x_train2_fld, x_test2_fold = X2_train.iloc[train_index], X2_train.iloc[test_index]
    y_train_fold, y_test_fold = y_train.iloc[train_index], y_train.iloc[test_index]

    #compute the class weights
    class_weights = compute_class_weight('balanced', np.unique(y_train_fold),y = np.ravel(y_train_fold))

    class_weights_list.append(class_weights)

    ncce = weighted_categorical_crossentropy(weights=np.array(class_weights))

    model = getModel(maxlen, embedding_matrix)

    model.compile(loss=ncce, optimizer='adam', metrics=['acc'])

    model.fit(x=[x_train1_fold, x_train2_fld], y=y_train_fold, batch_size=10, epochs=35, verbose=1)

    score = model.evaluate(x=[x_test1_fold, x_test2_fold], y=y_test_fold, verbose=0)

    print("Accuracy Validation: %.2f%%" % (score[1]*100))

In [ ]:
ncce = weighted_categorical_crossentropy(weights=np.array(class_weights_list[best_class_weight]))

In [ ]:
model = getModel(maxlen, embedding_matrix)

In [ ]:
model.compile(loss=ncce, optimizer='adam', metrics=['acc'])

In [ ]:
history = model.fit(x=[X1_train, X2_train], y=y_train, batch_size=10, epochs=35, verbose=1)

In [ ]:
score = model.evaluate(x=[X1_test, X2_test], y=y_test, verbose=0)

print("Accuracy Test: %.2f%%" % (score[1]*100))

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

y_pred = model.predict(x=[X1_test, X2_test])

y_pred_clean = np.zeros_like(y_pred)
for idx, i in enumerate(np.argmax(y_pred,axis=1)):
    y_pred_clean[idx][i] = 1

print(classification_report(y_test, y_pred_clean))